# Step 1: Data and Model Preparation


In [1]:
exp_name = 'exp_058'

In [2]:
%load_ext autoreload
%autoreload 2
import sys
import os
codebase = '../../'
sys.path.append(codebase)

In [3]:
from src.dataflow.vg_graph_loader_3 import VGGraphLoader
from src.dataflow.graph_batcher_2_rpn import GraphBatcher

In [4]:
from src.tensorflow.models.vsp_model_002 import Model
from src.tensorflow.engine.trainer_3 import Trainer
from src.tensorflow.engine.util import deploy, load_params
from src.util.util import pickle_save, pickle_load
from src.eval.sgg_evaluator_03 import Evaluator


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [5]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(1224)
np.random.seed(1225)

In [6]:
gl = VGGraphLoader(
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'hanwang', 'sg.pkl'),
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'hanwang', 'class_embs.pkl'),
)


In [7]:
img_ids, train_idx, test_idx = pickle_load(os.path.join(codebase, 'metadata', 'VG', 'prep', 'hanwang', 'split.pkl'))


In [8]:
np.random.shuffle(train_idx)
val_idx = train_idx[:1000]
train_idx = train_idx[1000:]

In [9]:
proposals = pickle_load(os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'coords.pkl'))

In [10]:
batcher = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1236, 
)
batcher.set_subset(train_idx)
batcher.shuffle()

In [11]:
batcher_val = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1237, 
)
batcher_val.set_subset(val_idx)

In [12]:
tf.reset_default_graph()

In [12]:
model = Model(   
    dim_att_head_role_ent=[1024, 1024], 
    dim_att_head_role_pred=[1024, 1024],
    role_edge_mode='full_soft',
    stop_grad_role_edge=True,
    
    dim_state_ent=1024,
    dim_state_pred=1024,
    dim_emb_head_ent=[300],
    dim_conf_head_ent=[],
    dim_conf_head_pred=[],
    dim_emb_head_pred=[300],
    dim_init_head_ent=[1024],

    dim_message_send_head_pred2ent=[1024, 1024],
    dim_message_pool_head_pred2ent=[1024, 1024],
    dim_message_receive_head_pred2ent=[1024, 1024],    
    dim_message_send_head_ent2pred=[1024, 1024],
    dim_message_pool_head_ent2pred=[1024, 1024],
    dim_message_receive_head_ent2pred=[1024, 1024],
    
    num_mp_iter=1,     
    num_proposals=None,

    heat_role=1.0,
    heat_emb_mil=1.0,
    null_att_logit=0.0,
        
    num_align_iter=3,
    alignment_order='best',
    default_act_fn_name='leaky_relu',
    optimizer_type_generator=tf.train.AdamOptimizer,
    
    max_num_pred=100,
    dim_init_head_pred=[],
    init_state_type_pred='trainable',
    
    dim_proposal_feat=1536,
    emb_dict_ent=gl.noun_emb_dict,
    emb_dict_pred=gl.pred_emb_dict,    
    #image_size=448,
    #num_channels=3,
    debugging=False
)



Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
Instructions for updating:
Use tf.cast instead.


In [13]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True)))
#sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5)))
sess = tf.Session()
sess.run(tf.no_op())

# Step 2: Training


In [14]:
train_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 20,
    'filter_nonoverlap': True
})
val_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 20,
    'filter_nonoverlap': True
})


In [15]:
trainer = Trainer(
    model, sess, batcher, batcher_val, train_evaluator, val_evaluator,
    logdir=os.path.join(codebase, 'log', 'tfsummary', exp_name), 
    modeldir=os.path.join(codebase, 'checkpoint', exp_name),
)

In [16]:
trainer.train (
    max_steps=30000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=10000, 
    hyperparams = {
        'learning_rate_generator': 1e-5 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 0.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


Step 1000: loss = 5.191999912261963 (2822.872 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.373, 'PerImagePredMR': 0.941, 'PerImageOneHopMP_at100_iou_0.5': 0.002, 'PerImageOneHopMR_at100_iou_0.5': 0.01, 'PerImageOneHopMP_at50_iou_0.5': 0.002, 'PerImageOneHopMR_at50_iou_0.5': 0.008, 'PerImageOneHopMP_phrase_at100_iou_0.5': 0.015, 'PerImageOneHopMR_phrase_at100_iou_0.5': 0.075, 'PerImageOneHopMP_phrase_at50_iou_0.5': 0.022, 'PerImageOneHopMR_phrase_at50_iou_0.5': 0.061}
Validation Results: {'PerImagePredMP': 0.385, 'PerImagePredMR': 0.937, 'PerImageOneHopMP_at100_iou_0.5': 0.002, 'PerImageOneHopMR_at100_iou_0.5': 0.013, 'PerImageOneHopMP_at50_iou_0.5': 0.002, 'PerImageOneHopMR_at50_iou_0.5': 0.01, 'PerImageOneHopMP_phrase_at100_iou_0.5': 0.015, 'PerImageOneHopMR_phrase_at100_iou_0.5': 0.081, 'PerImageOneHopMP_phrase_at50_iou_0.5': 0.022, 'PerImageOneHopMR_phrase_at50_iou_0.5': 0.068}
Step 2000: loss = 5.589000225067139 (5500.059 sec)
Training Batch Evaluation Results: {'Pe

In [ ]:
trainer.train (
    max_steps=10000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=1000, 
    hyperparams = {
        'learning_rate_generator': 1e-6 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 0.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


Step 31000: loss = 4.63100004196167 (2935.506 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.527, 'PerImagePredMR': 0.916, 'PerImageOneHopMP_at100_iou_0.5': 0.005, 'PerImageOneHopMR_at100_iou_0.5': 0.029, 'PerImageOneHopMP_at50_iou_0.5': 0.008, 'PerImageOneHopMR_at50_iou_0.5': 0.026, 'PerImageOneHopMP_phrase_at100_iou_0.5': 0.033, 'PerImageOneHopMR_phrase_at100_iou_0.5': 0.182, 'PerImageOneHopMP_phrase_at50_iou_0.5': 0.052, 'PerImageOneHopMR_phrase_at50_iou_0.5': 0.155}
Validation Results: {'PerImagePredMP': 0.517, 'PerImagePredMR': 0.905, 'PerImageOneHopMP_at100_iou_0.5': 0.005, 'PerImageOneHopMR_at100_iou_0.5': 0.032, 'PerImageOneHopMP_at50_iou_0.5': 0.008, 'PerImageOneHopMR_at50_iou_0.5': 0.029, 'PerImageOneHopMP_phrase_at100_iou_0.5': 0.036, 'PerImageOneHopMR_phrase_at100_iou_0.5': 0.187, 'PerImageOneHopMP_phrase_at50_iou_0.5': 0.056, 'PerImageOneHopMR_phrase_at50_iou_0.5': 0.156}
Step 32000: loss = 3.9649999141693115 (5797.875 sec)
Training Batch Evaluation Results: 

In [21]:
trainer.train (
    max_steps=5000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=1000, 
    hyperparams = {
        'learning_rate_generator': 1e-7 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 0.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


Step 41000: loss = 3.8289999961853027 (3041.728 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.517, 'PerImagePredMR': 0.933, 'PerImageOneHopMP_at100_iou_0.5': 0.005, 'PerImageOneHopMR_at100_iou_0.5': 0.031, 'PerImageOneHopMP_at50_iou_0.5': 0.009, 'PerImageOneHopMR_at50_iou_0.5': 0.027, 'PerImageOneHopMP_phrase_at100_iou_0.5': 0.036, 'PerImageOneHopMR_phrase_at100_iou_0.5': 0.196, 'PerImageOneHopMP_phrase_at50_iou_0.5': 0.056, 'PerImageOneHopMR_phrase_at50_iou_0.5': 0.166}
Validation Results: {'PerImagePredMP': 0.51, 'PerImagePredMR': 0.91, 'PerImageOneHopMP_at100_iou_0.5': 0.005, 'PerImageOneHopMR_at100_iou_0.5': 0.033, 'PerImageOneHopMP_at50_iou_0.5': 0.009, 'PerImageOneHopMR_at50_iou_0.5': 0.028, 'PerImageOneHopMP_phrase_at100_iou_0.5': 0.036, 'PerImageOneHopMR_phrase_at100_iou_0.5': 0.195, 'PerImageOneHopMP_phrase_at50_iou_0.5': 0.057, 'PerImageOneHopMR_phrase_at50_iou_0.5': 0.163}
Step 42000: loss = 3.632999897003174 (6004.474 sec)
Training Batch Evaluation Results: {

In [ ]:
trainer.save_model('final_1')

# Step 3: Evaluation

In [14]:
load_params(sess, os.path.join(codebase, 'checkpoint', exp_name, 'ckpt-45000'))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../../checkpoint/exp_058/ckpt-45000


In [15]:
test_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 20,
    'filter_nonoverlap': True
})


In [16]:
batcher_test = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1237, 
)
batcher_test.set_subset(test_idx)

In [17]:
#test_evaluator.reset()
deploy(sess, model, batcher_test, test_evaluator, model.eval_ops,
    hyperparams={
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
    ignore_errors=False,
    debug_mode=False,
    verbose_rate=100
)


Deploying
Batch 1/808 done in 0 seconds. Spent time for read=0.01 (avg 0.01), processing=0.46 (avg 0.46), write=0.00 (avg 0.00) seconds.
Batch 101/808 done in 3 seconds. Spent time for read=0.01 (avg 0.01), processing=0.02 (avg 0.02), write=0.00 (avg 0.00) seconds.
Batch 201/808 done in 5 seconds. Spent time for read=0.01 (avg 0.01), processing=0.02 (avg 0.02), write=0.00 (avg 0.00) seconds.
Batch 301/808 done in 8 seconds. Spent time for read=0.01 (avg 0.01), processing=0.02 (avg 0.02), write=0.00 (avg 0.00) seconds.
Batch 401/808 done in 11 seconds. Spent time for read=0.01 (avg 0.01), processing=0.02 (avg 0.02), write=0.00 (avg 0.00) seconds.
Batch 501/808 done in 14 seconds. Spent time for read=0.01 (avg 0.01), processing=0.02 (avg 0.02), write=0.00 (avg 0.00) seconds.
Batch 601/808 done in 16 seconds. Spent time for read=0.01 (avg 0.01), processing=0.02 (avg 0.02), write=0.00 (avg 0.00) seconds.
Batch 701/808 done in 19 seconds. Spent time for read=0.00 (avg 0.01), processing=0.02

In [26]:
test_results = test_evaluator.evaluate()
test_results

{'PerImagePredMP': 0.5138429704119125,
 'PerImagePredMR': 0.9163794513859109,
 'PerImageOneHopMP_at100_iou_0.5': 0.00441771417520789,
 'PerImageOneHopMR_at100_iou_0.5': 0.027695302720784038,
 'PerImageOneHopMP_at50_iou_0.5': 0.007591568362018952,
 'PerImageOneHopMR_at50_iou_0.5': 0.02450071425051161,
 'PerImageOneHopMP_phrase_at100_iou_0.5': 0.03403751692129182,
 'PerImageOneHopMR_phrase_at100_iou_0.5': 0.18278795717670876,
 'PerImageOneHopMP_phrase_at50_iou_0.5': 0.05326551924192613,
 'PerImageOneHopMR_phrase_at50_iou_0.5': 0.15450709790253925}